In [ ]:
#%run retropy.ipynb

In [ ]:
def parseTargets(targets):
    # single account
    if isAccount(targets):
        account = targets[0]
        symbols = get_holdings(account).index.values
    # entire portfolio
    elif len(targets) == 0 or (len(targets) == 1 and targets[0] == ""):
        symbols = df.symbol.dropna().unique()
        account = None
    # single symbol or list of symbols
    else:
        symbols = []
        for s in targets:
            if isinstance(s, str):
                symbols.append(s)
            elif isinstance(s, list):
                symbols += s
            else:
                raise Exception("Unsupported target type {0} of {1}".format(type(s), s))
            
        account = None
    return account, symbols

def get_targets_name(targets):
    account, symbols = parseTargets(targets)
    if account: return account
    return "[" + (", ".join(symbols)) + "]"


class Targets:
    def __init__(self, *targets, start=None, end=None):
        self.account, self.symbols = parseTargets(targets)
        self.start = date(start) if start else None
        self.end = date(end) if end else None
        if len(targets) == 0 or (len(targets) == 1 and targets[0] == ""):
            self.name = "Total"
        else:
            self.name = get_targets_name(targets)

def filter_targets(targets: Targets):
    if targets.account:
        res = df[df.to == targets.account]
    elif len(targets.symbols) == 0:
        res = df
    else:
        res = df[df.symbol.isin(targets.symbols)]
    if targets.start:
        res = df[df.date >= targets.start]
    if targets.end:
        res = df[df.date <= targets.start]
    return res

def asTargets(x):
    if isinstance(x, Targets): return x
    return Targets(x)

In [ ]:
def df_search(text):
    df[df.apply(str, axis=1).str.contains(text)]
    

def get_holdings(account, showall=True, start=None, end=None, exclude=None):
    tmp = df[(df["to"] == account) & (df["units"] != 0)] if account else df
    if start:
        tmp = tmp[tmp.date >= pd.to_datetime(start)]
    if end:
        tmp = tmp[tmp.date <= pd.to_datetime(end)]
    gby = tmp = tmp.groupby("symbol")
    tmp = gby["units"].sum().to_frame()
    tmp["first"] = gby.first().date
    tmp["last"] = gby.last().date
    if exclude:
        tmp = tmp[~tmp.index.isin(exclude)]
    #tmp = tmp.groupby("symbol")["units"].sum()
    if not showall:
        tmp = tmp[tmp.units > 0]
    return tmp

#def get_flow(account):
#    tmp = df[(df["to"] == account) & (df["units"] != 0)]
#    resFrom = tmp.groupby("date")["from_amount"].sum().to_frame()
#    resTo   = tmp.groupby("date")["to_amount"].sum().to_frame()
#    res = merge(resFrom, resTo)
#    return res

def get_flow(targets: Targets, flatten=True):
    tmp = filter_targets(targets)
    tmp = tmp.groupby(["date", "from_cur"])["from_amount"].sum()
    tmp = tmp.unstack(level=1)
    if flatten:
        for c in tmp:
            if c == "USD":
                continue
            forex = getForex(c, "USD")
            tmp[c] = tmp[c] * forex
        tmp = tmp.sum(axis=1)
    return tmp

def get_holdings_changes(account, datesOnly=True):
    tmp = df[(df["to"] == account) & (df["units"] != 0)]
    tmp = tmp.groupby(["symbol", "date"])
    tmp = tmp.units.sum().unstack(level=0).fillna(0)
    tmp = tmp.sort_index()
    tmp = tmp.apply(lambda x: x.cumsum())
    tmp = tmp.apply(lambda x: "|".join(x[x>0].index.values), axis=1).drop_duplicates()
    if datesOnly:
        tmp = pd.to_datetime(tmp.index)#.to_datetime()
    return tmp
    
#get_flow("TAA", flatten=False)
#a = "TAA"

def get_units_per_place_and_symbol(*targets):
    tmp = filter_targets(targets)
    tmp = tmp.groupby(["symbol", "date", "place"]).units.sum().to_frame().unstack(level=1).T
    return tmp

#get_units_per_place_and_symbol(["NB.TO!CAD", "NIOBF"])

def isAccount(targets):
    global accounts
    if isinstance(targets, str):
        return targets in accounts
    return len(targets) == 1 and isinstance(targets[0], str) and targets[0] in accounts


In [ ]:
# less friendly way
def get_cash_balance(flatten=True):
    _from = df[df["from"].str.startswith("CASH")].groupby(["from", "from_cur"])["from_amount"].sum()
    _to   = df[df["to"].str.startswith("CASH")].groupby(["to", "to_cur"])["to_amount"].sum()
    _from.index.rename(["account", "cur"], inplace=True)
    _to.index.rename(["account", "cur"], inplace=True)
    cash_flow = _to.sub(_from, fill_value=0)
    #cash_flow
    if flatten:
        cash_flow = cash_flow.groupby("cur").sum()
    return cash_flow

# this way is simple and fast, but won't support cases where the place in "to" and "from" are different
def get_cash_balance(flatten=True):
    fromPT = pd.pivot_table(df[df["from"].str.startswith("CASH")], index="place", columns="from_cur", values="from_amount", aggfunc="sum", margins=True, margins_name="_Total")
    toPT = pd.pivot_table(df[df["to"].str.startswith("CASH")], index="place", columns="to_cur", values="to_amount", aggfunc="sum", margins=True, margins_name="_Total")
    cf = toPT.sub(fromPT, fill_value=0)
    return cf.drop("_Total", axis=1)

def cash_place(s):
    if not s.startswith("CASH-"):
        return "NA"
    s = s.replace("CASH-", "")
    return "-".join(s.split("-")[:-1])

# this way is slower, but DOES support cases where the place in "to" and "from" are different
def get_cash_balance(flat=False):
    tmp = df.assign(to_place=df.to.apply(cash_place)).assign(from_place=df["from"].apply(cash_place))
    fromPT = pd.pivot_table(tmp[tmp["from"].str.startswith("CASH")], index="from_place", columns="from_cur", values="from_amount", aggfunc="sum", margins=True, margins_name="_Total")
    toPT = pd.pivot_table(tmp[tmp["to"].str.startswith("CASH")], index="to_place", columns="to_cur", values="to_amount", aggfunc="sum", margins=True, margins_name="_Total")
    res = toPT.sub(fromPT, fill_value=0)
    res = res.drop("_Total", axis=1)
    if flat:
        res = res.sum()
    return res

# def get_cash_type_summary():
#     _from = df[df["from"].str.startswith("CASH")].groupby(["type", "from_cur"])["from_amount"].sum()
#     _to   = df[df["to"].str.startswith("CASH")].groupby(["type", "to_cur"])["to_amount"].sum()
#     _from.index.rename(["type", "cur"], inplace=True)
#     _to.index.rename(["type", "cur"], inplace=True)
#     cash_flow = _to.sub(_from, fill_value=0)
#     return cash_flow.unstack()
    
def get_cash_type_summary():
    fromPT = pd.pivot_table(df[df["from"].str.startswith("CASH")], index="type", columns="from_cur", values="from_amount", aggfunc="sum", margins=True, margins_name="_Total")
    toPT = pd.pivot_table(df[df["to"].str.startswith("CASH")], index="type", columns="to_cur", values="to_amount", aggfunc="sum", margins=True, margins_name="_Total")
    cf = toPT.sub(fromPT, fill_value=0)
    return cf.drop("_Total", axis=1)


In [ ]:
#fixSymbols = None
#ignoredAssets = None
#accounts = None

def validate(df):
    assert np.sum(df.place.isnull()) == 0, "some rows have undefined 'place' field"

def init_log_parser(filename: str, _fixSymbols, _ignoredAssets):
    global accounts, fixSymbols, ignoredAssets
    fixSymbols = _fixSymbols
    conf.ignoredAssets += _ignoredAssets
    df = load_log(filename)
    validate(df)
    accounts = set([x for x in df["to"].unique() if not x.startswith("CASH") and not x in ["VOID", "*", ""]])
    print(accounts)
    return df
    
def load_log(filename: str):
    df = pd.read_csv("log.tsv", sep="\t", na_values=["-", "???"])
    df["Date"] = pd.to_datetime(df["Date"], format="%m/%d/%Y")
    df.columns=["date", "type", "from", "from_cur", "from_amount", "to_amount", "to_cur", "to", "symbol", "units", "price", "fee", "place", "comments", "_junk"]
    df["units"] = series_as_float(df.units.str.replace(",", "")).fillna(0)
    df["price"] = series_as_float(df.price.str.replace(",", "").replace("\$", "", regex=True)).fillna(0)
    df["from_amount"] = series_as_float(df.from_amount.str.replace(",", "")).fillna(0)
    df["to_amount"] = series_as_float(df.to_amount.str.replace(",", "")).fillna(0)
    df["fee"] = series_as_float(df.fee.str.replace(",", "").replace("\$", "", regex=True)).fillna(0)
    df["comments"] = df["comments"].fillna("").astype("str")
    df["from"] = df["from"].fillna("")
    df["to"] = df["to"].fillna("")

    print(fixSymbols)
    for k in fixSymbols:
        df["symbol"] = df.symbol.replace(k, fixSymbols[k])
    
    #patch
    #crypto = np.setdiff1d(df.query("to=='CRYPTO'").symbol.unique(), ["0LNB", "0LND"])
    #for sym in crypto:
    #    df["symbol"] = df.symbol.replace(sym, sym+"@CC")

    df.sort_values("date", inplace=True)    
    
    return df

In [ ]:
# TODO: fix
def get_curr_value(holdings):
    holdings = holdings.copy()
    holdings.columns = ["units"]
    holdings["price"] = holdings.index.map(lambda x: curr_price(x))
    holdings["currency"] = holdings.index.map(symbolCurrency)
    holdings["value"] = holdings["price"] * holdings["units"]
    if len(holdings) > 0:
        holdings["value"] = holdings.apply(lambda x: convert(x["value"], x["currency"], "USD"), axis=1)
    return holdings

if False:
    for acc in accounts:
        print(acc)
        holdings = get_holdings(acc)
        print(get_value(holdings)["value"].sum())
    
    
#get_value(get_holdings("")).value.sum()

In [ ]:
def getFxRate(fromCur, toCur, date):
    #print(fromCur, toCur, date)
    if fromCur == toCur:
        return 1
    fx = getForex(fromCur, toCur)
    return fx[date]

def usdify(val, cur):
    df = pd.DataFrame({"val": val, "cur": cur})
    #df["rate"] = df.apply(lambda x: print(type(x.name)), axis=1)
    df["rate"] = df.apply(lambda x: getFxRate(x.cur, "USD", x.name), axis=1)
    return df.val * df.rate
    


def get_symbol_history(sym, account=None, endtoday=True):
    if account:
        cumUnits = df[(df.symbol == sym) & (df.to == account)].copy()
    else:
        cumUnits = df[df.symbol == sym].copy()
    cumUnits["cunits"] = cumUnits.units.cumsum()
    cumUnits = cumUnits.groupby("date")["cunits"].last() # we do this to have a single date in the index
    assert len(cumUnits) > 0, "no units history for {0} in account [{1}]".format(sym, account)
    start = cumUnits.index.min()
    end = datetime.date.today() if endtoday else res.index[-1]

    # we get the raw unadjusted price of the symbol (any splits will be reflected in the units count)
    symVal = get(sym, adj=False).to_frame().rename({sym: "price"}, axis=1)
    
    # outer join merge prices and units, as they may have different dates
    res = merge(symVal, cumUnits.to_frame(), ["price", "units"])

    # reindex so that we'll have a row for every date (this is required to easily apply operations between different symbols)
    #dr = pd.date_range(start = res.index[0], end=res.index[-1])
    dr = pd.date_range(start = start, end=end) # we want to start from the first date we have units for this symbol
    res = res.reindex(dr)
    
    _symbol = Symbol(sym)
    if _symbol.currency:
        forex = getForex(_symbol.currency, "USD")
        res["exrate"] = forex

    # fill any dates with data gaps using fill-forward
    res = res.fillna(method="ffill")
    
    # when we have an executed price of a symbol, we use it instead of the "market price"
    # most of the time it doesn't make a difference, but in 2018-02-05 SVXY crash it makes a huge difference
    execPrice = df[df.symbol == sym].copy()
    #execPrice = execPrice.groupby("date")["price"].last().replace(0, np.nan)
    execPrice = execPrice.groupby("date")["price", "to_cur"].last().replace(0, np.nan)
    if not _symbol.currency: # we must be careful, not to convert currencies twice
        execPrice = usdify(execPrice.price, execPrice.to_cur.fillna("USD"))
    else:
        execPrice = execPrice.price
    
    res["execPrice"] = execPrice
    res["price"] = res.execPrice.fillna(res.price)
    res = res.drop("execPrice", axis=1)

    # fill any dates with data gaps using fill-forward, again
    res = res.fillna(method="ffill")
    
    if _symbol.currency:
        res["value"] = res.units * res.price * res.exrate
    else:
        res["value"] = res.units * res.price

    #res = res.dropna(how='all') # get rid of rows which have no data at all

    return res

def get_portfolio_value(targets: Targets, split=False):
    targets = asTargets(targets)
    d = dict(map(lambda x: (x, get_symbol_history(x, targets.account).value), targets.symbols))
    #pdf = pd.DataFrame.from_dict(d, orient='index').T
    pdf = pd_from_dict(d)
    pdf = pdf.sort_index()
    
    if split:
        # show each holding seperatly
        #l = [pdf[c] for c in pdf]
        l = [pdf[c] for c in pdf]
        #l = [s.replace(0, np.nan) for s in l]
        l = [s.fillna(0) for s in l]
        #l = [s[s.first_valid_index():] for s in l]
        l = sorted(l, key=lambda x: x.index[0])
        return pd.DataFrame(l).T
    else:
        # sum all, and drop dates with zero value
        value = pdf.sum(axis=1).replace(0, np.nan).dropna()
        return value

def show_value(targets: Targets, split=False, skip_split_if_single=True):
    v = get_portfolio_value(targets, split=split)
    if split:
        if skip_split_if_single and v.shape[1] == 1:
            print("show_value(split=True) skipped, has only one asset: " + v.columns.values[0])
        show([v[s] for s in v], legend=len(v)<10)
    else:
        show(v)
    
def show_symbol(sym):
    #show(get_symbol_history("BWX").value.replace(0, np.nan).dropna())
    show(get_symbol_history(sym).value)


def merge(a, b, names):
    if isinstance(a, pd.Series):
        a = a.to_frame()
    if isinstance(b, pd.Series):
        b = b.to_frame()
    tmp = pd.merge(a, b, how="outer", left_index=True, right_index=True)
    tmp.columns = names
    return tmp

def get_portfolio_pnl(targets: Targets, base_cur=None):
    targets = asTargets(targets)
    pv = get_portfolio_value(targets)
    pf = get_flow(targets)
    tmp = merge(pv, pf, ["value", "flow"]).fillna(0)

    if base_cur:
        tmp["rate"] = getForex("USD", base_cur)
        tmp["flow"] = tmp.flow * tmp.rate
        tmp["value"] = tmp.value * tmp.rate
    
    tmp["flow"] = tmp.flow.cumsum()
    tmp["P&L"] = tmp.value - tmp.flow
    tmp["gain"] = tmp["P&L"] / tmp.flow + 1
    
    # when the value drops to zero (portfolio closed), replace with nan so that we have a clear cutoff in the charts
    tmp.loc[tmp.value == 0, :] = np.nan
    
    return tmp

def get_portfolio_gain(targets: Targets):
    targets = asTargets(targets)
    tmp = get_portfolio_pnl(targets)
    return wrap(tmp["gain"], targets.name + " gain")

def show_portfolio_pnl(targets: Targets, base_cur=None):
    targets = asTargets(targets)
    tmp = get_portfolio_pnl(targets, base_cur)
    title = targets.name + " P&L"
    if base_cur:
        title += " in " + base_cur
    if targets.account:
        changes = get_holdings_changes(targets.account)
        show(tmp, *changes, log=False, title=title)
    else:
        show(tmp, log=False, title=title)

def irr_helper(df):
    x = -df.flow.dropna()
    lastValue = df.value[-1]
    if lastValue <= 0:
        irr = 0
    else:
        lastDate = pd.to_datetime(df.index)[-1]#.to_datetime()[-1]
        try:
            values = np.append(x.values, [lastValue])
            dates = list(pd.to_datetime(x.index)) + [lastDate]
            #print(values)
            #print(dates)
            irr = xirr(values, dates)
        except:
            irr = 0
    days = (df.index[-1] - df.index[0]).days
    years = days / 365.0
    res = (1+irr) ** years
    if False and res < 0.5:
        xx = pd.DataFrame({"v": values, "d": dates})
        display(xx)
        print(values)
        print(dates)
        print("res: ", res)
        print("irr: ", irr)
        assert False
    return res
        
def get_portfolio_irr(targets: Targets):
    targets = asTargets(targets)
    pv = get_portfolio_value(targets)
    pf = get_flow(targets)
    tmp = merge(pv, pf, ["value", "flow"])
    
    agg = pd.DataFrame(columns=["value", "flow"])
    res = pd.Series()
    for k, v in tmp.iterrows():
        agg.loc[k,:] = v.values
        res[k] = irr_helper(agg)
    return wrap(res, targets.name + " IRR")
        #print(agg)
    #return tmp.expanding().apply(irr_helper)
    #return tmp.expanding().apply(lambda x: x.sum())
    
    #pf[pv.index[-1]] = -pv[-1]
    #return xirr(pf.values, pf.index.to_datetime())*100
#show(port({"VT": 60, "AGG": 40}, "bench"))

def show_portfolio_performance(targets: Targets, *extra, **showArgs):
    targets = asTargets(targets)
    irr = get_portfolio_irr(targets)
    gain = get_portfolio_gain(targets)
    base = 1
    if not targets.account and len(targets.symbols) == 1:
        base = get(targets.symbols[0])
    show(gain, irr, base, 1, *extra, log=False, **showArgs)
    
def get_asset_allocation(targets: Targets, kind="value"):
    targets = asTargets(targets)
    tmp = filter_targets(targets)
    
    # map given symbols to asset-classes and collect for use
    acTmp = {}
    for s in tmp.symbol.dropna().unique():
        ac = assetClassesMap.get(s, "UNKNOWN")
        if not ac in acTmp: 
            acTmp[ac] = []
        acTmp[ac].append(s)

    # calc historic value of each collected asset-class
    d = {}
    for ac in acTmp:
        symbols = acTmp[ac]
        p = Targets(symbols)
        value = d[ac] = get_by_kind(p, kind)
        d[ac] = value
        
    # join in a data-frame
    #res = pd.DataFrame.from_dict(d, orient='index').T.fillna(0)
    res = pd_from_dict(d).fillna(0)
    return res

def show_asset_allocation(targets: Targets, kind="value", skip_if_single=True):
    targets = asTargets(targets)
    
    if kind == "all":
        show_asset_allocation(targets, kind="value", skip_if_single=skip_if_single)
        show_asset_allocation(targets, kind="flow", skip_if_single=skip_if_single)
        show_asset_allocation(targets, kind="profit", skip_if_single=skip_if_single)
        show_asset_allocation(targets, kind="loss", skip_if_single=skip_if_single)
        return
    
    tmp = get_asset_allocation(targets, kind=kind)
    if skip_if_single and tmp.shape[1] == 1:
        print("show_asset_allocation skipped, has only one asset class: " + tmp.columns.values[0])
        return
    plotly_area(tmp, title=targets.name + " " + kind)

def get_by_kind(ac, kind):
    if kind == "value":
        return get_portfolio_value(ac)
    else:
        pnl = get_portfolio_pnl(ac)
        if kind == "flow":
            return pnl.flow
        elif kind == "P&L":
            return pnl["P&L"]
        elif kind == "profit":
            return np.maximum(pnl["P&L"], 0)
        elif kind == "loss":
            return -np.minimum(pnl["P&L"], 0)
        else:
            raise Exception("unsupported kind: " + kind)
    
def get_portfolio_allocation(kind="value", exclude=None):
    d = {}
    for ac in accounts:
        if exclude and ac in exclude:
            continue
        d[ac] = get_by_kind(ac, kind)
    return pd_from_dict(d).fillna(0)

def show_portfolio_allocation(kind="value", exclude=None):
    if kind == "all":
        show_portfolio_allocation(kind="value")
        show_portfolio_allocation(kind="flow")
        show_portfolio_allocation(kind="profit")
        show_portfolio_allocation(kind="loss")
        return
    
    plotly_area(get_portfolio_allocation(kind, exclude), title=kind)
    
def show_portfolio(p, bench="SPY", bench2=None, **show_args):
    irr = get_portfolio_irr(p)
    all_bench = []
    if not bench is None: all_bench.append(get(bench))
    if not bench2 is None: all_bench.append(get(bench2))
    all_bench_div = [irr/bench for bench in all_bench]
    show(irr, *all_bench, *all_bench_div, 1, **show_args)
    show_portfolio_pnl(p)
    show_asset_allocation(p)
    
    dds = doTrim([dd(s) for s in ([irr] + all_bench)])
    show(*dds, log=False)
    
    show_value(p, split=True)
    